In [2]:
import time
import numpy as np
import tensorflow as tf

In [11]:
class Model:
    
    def __init__(self):
        
        tf.reset_default_graph()
        
        (x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = tf.keras.datasets.mnist.load_data()
        
        #Flatten the training and test images
        x_train_flat = x_train_orig.reshape(x_train_orig.shape[0], -1)
        x_test_flat = x_test_orig.reshape(x_test_orig.shape[0], -1)
        
        (self.m, self.n_x) = x_train_flat.shape
        self.n_y = 10  
        
        #Normalize image vectors
        self.x_train = x_train_flat / np.float32(255)
        self.x_test = x_test_flat / np.float32(255)        
            
        #Convert training and test labels to on hot matrices
        #self.y_train = self.one_hot(y_train_orig, self.n_y)
        #self.y_test = self.one_hot(y_test_orig, self.n_y)                     
           
    #def one_hot(self, labels, numclasses):
    #    return np.eye(numclasses)[labels]
    
    #def create_variables(self):
    #    with tf.variable_scope("variables"):
    #        W1 = tf.get_variable("W1", [self.n_x, 500], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #        b1 = tf.get_variable("b1", [1, 500], initializer = tf.zeros_initializer())
    #        W2 = tf.get_variable("W2", [500, 500], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #        b2 = tf.get_variable("b2", [1, 500], initializer = tf.zeros_initializer())
    #        W3 = tf.get_variable("W3", [500, 10], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    #        b3 = tf.get_variable("b3", [1, 10], initializer = tf.zeros_initializer()) 
        
    #Test
    def fully_connected(self, A_prev, nbr_inputs, nbr_outputs):
        W = tf.get_variable("W", [nbr_inputs, nbr_outputs], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        b = tf.get_variable("b", [1, nbr_outputs], initializer = tf.zeros_initializer())            
        Z = tf.add(tf.matmul(A_prev, W), b)
        A = tf.nn.relu(Z)
        return A
        
    def forward_propagation(self, X):
        with tf.variable_scope("hidden1", reuse=tf.AUTO_REUSE):
            A1 = self.fully_connected(X, 784, 500)
        with tf.variable_scope("hidden2", reuse=tf.AUTO_REUSE):
            A2 = self.fully_connected(A1, 500, 2)
        with tf.variable_scope("hidden3", reuse=tf.AUTO_REUSE):
            A3 = self.fully_connected(A2, 2, 500)
        with tf.variable_scope("outputs", reuse=tf.AUTO_REUSE):
            A4 = self.fully_connected(A3, 500, 784)
            return A4
        
    #def forward_propagation(self, X):
    #    with tf.variable_scope("variables", reuse=True):
    #        W1 = tf.get_variable("W1")
    #        b1 = tf.get_variable("b1")
    #        W2 = tf.get_variable("W2")
    #        b2 = tf.get_variable("b2")
    #        W3 = tf.get_variable("W3")
    #        b3 = tf.get_variable("b3") 
    #    
    #        Z1 = tf.add(tf.matmul(X, W1), b1)
    #        A1 = tf.nn.relu(Z1)
    #        Z2 = tf.add(tf.matmul(A1, W2), b2)
    #        A2 = tf.nn.relu(Z2)
    #        Z3 = tf.add(tf.matmul(A2, W3), b3)
    #        return Z3

    def compute_loss(self, predicted, actual):              
        squared_loss = tf.square(predicted - actual)
        reconstruction_loss = tf.reduce_mean(squared_loss)    
        return reconstruction_loss
    
    def create_optimizer(self, learning_rate):
        optimizer = tf.train.AdamOptimizer(learning_rate)
        return optimizer
    
    #def compute_accuracy(self, predicted, actual):
    #    correct_prediction = tf.equal(tf.argmax(predicted, axis=1), tf.argmax(actual, axis=1))
    #    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    #    return accuracy                 
        
    def train(self, num_epochs = 100):
    
        BATCH_SIZE = 1000
    
        tf.reset_default_graph()
        tf.set_random_seed(1)

        x = tf.placeholder(tf.float32, shape=[None, self.n_x])
        #y = tf.placeholder(tf.float32, shape=[None, self.n_y])

        # Define training dataset
        train_dataset = tf.data.Dataset.from_tensor_slices((x)).shuffle(self.m).batch(BATCH_SIZE).repeat()        
        train_iterator = train_dataset.make_initializable_iterator()        
        train_features = train_iterator.get_next()    
        train_init_op = train_iterator.make_initializer(train_dataset) 

        #self.create_variables()
        
        #Training
        train_predicted = self.forward_propagation(train_features)        
        loss = self.compute_loss(train_predicted, train_features)        
        optimizer = self.create_optimizer(0.001).minimize(loss)

        #Compute accuracy
        #test_predicted = self.forward_propagation(x)                
        #accuracy = self.compute_accuracy(test_predicted, y)
        
        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            sess.run(init)
            sess.run(train_init_op, feed_dict = {x : self.x_train})
                 
            nbrOfBatches = self.m // BATCH_SIZE

            begin_time = time.time()
            for epoch in range(num_epochs):

                epoch_loss = 0
                
                for _ in range(nbrOfBatches):
                    _, batch_loss = sess.run([optimizer, loss])
                    epoch_loss += batch_loss

                #train_accuracy = sess.run(accuracy, feed_dict = {x: self.x_train, y: self.y_train})
                #test_accuracy = sess.run(accuracy, feed_dict = {x: self.x_test, y: self.y_test})
                
                if (epoch % 10 == 0):
                    print("epoch: {}, train_loss: {:.4f}".format(
                        epoch, 
                        epoch_loss / nbrOfBatches
                    ))
                
            end_time = time.time()

            print('Model trained in {:.3f} (hh:mm:ss.ms)'.format(end_time - begin_time))
                     
            #train_accuracy = sess.run(accuracy, feed_dict = {x: self.x_train, y: self.y_train})
            #print("Accuracy on training set: {:.4f}".format(train_accuracy))
            #    
            #test_accuracy = sess.run(accuracy, feed_dict = {x: self.x_test, y: self.y_test})
            #print("Accuracy on test set: {:.4f}".format(test_accuracy))        

In [12]:
model = Model()
model.train()

epoch: 0, train_loss: 0.0798
epoch: 10, train_loss: 0.0540
epoch: 20, train_loss: 0.0500
epoch: 30, train_loss: 0.0481
epoch: 40, train_loss: 0.0470
epoch: 50, train_loss: 0.0462
epoch: 60, train_loss: 0.0456
epoch: 70, train_loss: 0.0449
epoch: 80, train_loss: 0.0446
epoch: 90, train_loss: 0.0440
Model trained in 421.170 (hh:mm:ss.ms)


In [ ]:
idx = 152
label = y_predict[idx].argmax()
image = x_test[idx].reshape(28,28)
plt.title('Example: %d  Label: %d' % (idx, label))
plt.imshow(image, cmap=plt.get_cmap('gray_r'))
plt.show()

In [13]:
(x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = tf.keras.datasets.mnist.load_data()
x_train_flat = x_train_orig.reshape(x_train_orig.shape[0], -1)
x_train = x_train_flat / np.float32(255)
reconstruction = model.forward_propagation(x_train[0, :])

ValueError: Shape must be rank 2 but is rank 1 for 'hidden1_1/MatMul' (op: 'MatMul') with input shapes: [784], [784,500].